# Baseline CNN Model

## Initial Setup

Auto update from code base

In [1]:
%load_ext autoreload
%autoreload 2

Import libraries

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
import copy
import json
import time

from src.model_api_pytorch import ModifiedCNNv2, train_model
from src.data import ProductDataset, pil_loader, make_submission

Read in datasets

In [3]:
df_beauty_train_raw = pd.read_csv('data/raw/beauty_data_info_train_competition.csv')
df_fashion_train_raw = pd.read_csv('data/raw/fashion_data_info_train_competition.csv')
df_mobile_train_raw = pd.read_csv('data/raw/mobile_data_info_train_competition.csv')

In [4]:
with open('data/raw/beauty_profile_train.json', 'r') as file:
    dict_beauty = json.load(file)
with open('data/raw/fashion_profile_train.json', 'r') as file:
    dict_fashion = json.load(file)
with open('data/raw/mobile_profile_train.json', 'r') as file:
    dict_mobile = json.load(file)

In [5]:
dict_mobile.keys()

dict_keys(['Operating System', 'Features', 'Network Connections', 'Memory RAM', 'Brand', 'Warranty Period', 'Storage Capacity', 'Color Family', 'Phone Model', 'Camera', 'Phone Screen Size'])

Remove NA and convert to integers

In [6]:
attributes_beauty = list(dict_beauty.keys())
attributes_fashion = list(dict_fashion.keys())
attributes_mobile = list(dict_mobile.keys())

df_beauty_train_preprocessed = df_beauty_train_raw.copy()
df_fashion_train_preprocessed = df_fashion_train_raw.copy()
df_mobile_train_preprocessed = df_mobile_train_raw.copy()

# df_beauty_train_preprocessed = df_beauty_train_preprocessed.dropna(how='all')
# df_fashion_train_preprocessed = df_fashion_train_preprocessed.dropna(how='all')
# df_mobile_train_preprocessed = df_mobile_train_preprocessed.dropna(how='all')


df_beauty_train_preprocessed[attributes_beauty] = df_beauty_train_preprocessed[attributes_beauty].fillna(999).astype(int)
df_fashion_train_preprocessed[attributes_fashion] = df_fashion_train_preprocessed[attributes_fashion].fillna(999).astype(int)
df_mobile_train_preprocessed[attributes_mobile] = df_mobile_train_preprocessed[attributes_mobile].fillna(999).astype(int)

Label encoding for attributes

In [7]:
# Beauty
le_beauty_list, encoded_beauty_list = list(), list()
for attribute in attributes_beauty:
    le_beauty_attr = LabelEncoder()
    encoded_beauty_attr = le_beauty_attr.fit_transform(df_beauty_train_preprocessed[attribute])
    le_beauty_list.append(le_beauty_attr)
    encoded_beauty_list.append(encoded_beauty_attr)
    
# Fashion
le_fashion_list, encoded_fashion_list = list(), list()
for attribute in attributes_fashion:
    le_fashion_attr = LabelEncoder()
    encoded_fashion_attr = le_fashion_attr.fit_transform(df_fashion_train_preprocessed[attribute])
    le_fashion_list.append(le_fashion_attr)
    encoded_fashion_list.append(encoded_fashion_attr)
    
# Mobile
le_mobile_list, encoded_mobile_list = list(), list()
for attribute in attributes_mobile:
    le_mobile_attr = LabelEncoder()
    encoded_mobile_attr = le_mobile_attr.fit_transform(df_mobile_train_preprocessed[attribute])
    le_mobile_list.append(le_mobile_attr)
    encoded_mobile_list.append(encoded_mobile_attr)

## Build Models

Num of classes for each attribute

In [8]:
num_classes_beauty = [len(le_beauty.classes_) for le_beauty in le_beauty_list]
print('Beauty:', num_classes_beauty)

num_classes_fashion = [len(le_fashion.classes_) for le_fashion in le_fashion_list]
print('Fashion:', num_classes_fashion)

num_classes_mobile = [len(le_mobile.classes_) for le_mobile in le_mobile_list]
print('Mobile:', num_classes_mobile)

Beauty: [8, 300, 40, 10, 9]
Fashion: [21, 17, 12, 20, 5]
Mobile: [8, 8, 5, 11, 56, 15, 9, 22, 619, 16, 7]


Transfer learning with ResNet18

In [9]:
## Beauty
model_beauty_ft = models.resnet18(pretrained=True)
# Freeze pre-trained layers
for param in model_beauty_ft.parameters():
    param.requires_grad = False
model_beauty_modified = ModifiedCNNv2(class_sizes=num_classes_beauty, attributes=attributes_beauty,
                                      model_pretrained=model_beauty_ft)
# print(model_beauty_modified)

## Fashion
model_fashion_ft = models.resnet18(pretrained=True)
# Freeze pre-trained layers
for param in model_fashion_ft.parameters():
    param.requires_grad = False
model_fashion_modified = ModifiedCNNv2(class_sizes=num_classes_fashion, attributes=attributes_fashion,
                                       model_pretrained=model_fashion_ft)
# print(model_fashion_modified)

## Mobile
model_mobile_ft = models.resnet18(pretrained=True)
# Freeze pre-trained layers
for param in model_mobile_ft.parameters():
    param.requires_grad = False
model_mobile_modified = ModifiedCNNv2(class_sizes=num_classes_mobile, attributes=attributes_mobile,
                                      model_pretrained=model_mobile_ft)
# print(model_fashion_modified)

Define device (CPU or CUDA), configure models for multiple GPUs and copy models to device

In [10]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model_beauty_modified.set_multiple_gpus()
model_beauty_modified.to(device)

model_fashion_modified.set_multiple_gpus()
model_fashion_modified.to(device)

model_mobile_modified.set_multiple_gpus()
model_mobile_modified.to(device)

ModifiedCNNv2(
  (model_pretrained): DataParallel(
    (module): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): 

## Train Models

Import dataset into dataloader

In [ ]:
transforms_pipeline = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [ ]:
dataset_beauty = ProductDataset(df=df_beauty_train_preprocessed, root_dir='data/raw',
                                encoded_attributes=encoded_beauty_list, transform=transforms_pipeline)

dataset_fashion = ProductDataset(df=df_fashion_train_preprocessed, root_dir='data/raw',
                                encoded_attributes=encoded_fashion_list, transform=transforms_pipeline)

dataset_mobile = ProductDataset(df=df_mobile_train_preprocessed, root_dir='data/raw',
                                encoded_attributes=encoded_mobile_list, transform=transforms_pipeline)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer_beauty = optim.Adam(model_beauty_modified.parameters(), lr=1e-3, weight_decay=0.01)
optimizer_fashion = optim.Adam(model_fashion_modified.parameters(), lr=1e-3, weight_decay=0.01)
optimizer_mobile = optim.Adam(model_mobile_modified.parameters(), lr=1e-3, weight_decay=0.01)

Beauty

In [14]:
model_beauty_trained = train_model(model=model_beauty_modified, dataset=dataset_beauty,
                                   criterion=criterion, optimizer=optimizer_beauty, device=device, num_epochs=3,
                                   batch_size=32, num_workers=12)
torch.save(model_beauty_trained.state_dict(), 'models/weights-cnn-baseline-beauty-v2.pt')

  0%|          | 0/8956 [00:00<?, ?it/s]

Epoch 1/3
----------


/home/yxlee245/ndsc-2019/src/model_api_pytorch.py:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  for c, attribute in zip(range(len(self.class_sizes)), self.attributes)}
  0%|          | 0/8956 [00:00<?, ?it/s]

Training Loss: 13.8791

Epoch 2/3
----------


  0%|          | 0/8956 [00:00<?, ?it/s]

Training Loss: 13.8712

Epoch 3/3
----------


100%|██████████| 8956/8956 [20:21<00:00, 12.45it/s]


Training Loss: 13.8692

Training completed in 61m 18s


Fashion

In [15]:
model_fashion_trained = train_model(model=model_fashion_modified, dataset=dataset_fashion,
                                    criterion=criterion, optimizer=optimizer_fashion, device=device, num_epochs=3,
                                    batch_size=32, num_workers=12)
torch.save(model_fashion_trained.state_dict(), 'models/weights-cnn-baseline-fashion-v2.pt')

  0%|          | 0/8599 [00:00<?, ?it/s]

Epoch 1/3
----------


  0%|          | 0/8599 [00:00<?, ?it/s]

Training Loss: 11.2597

Epoch 2/3
----------


  0%|          | 0/8599 [00:00<?, ?it/s]

Training Loss: 11.2378

Epoch 3/3
----------


100%|██████████| 8599/8599 [26:32<00:00,  5.40it/s]  


Training Loss: 11.2379

Training completed in 88m 11s


Mobile

In [16]:
model_mobile_trained = train_model(model=model_mobile_modified, dataset=dataset_mobile,
                                   criterion=criterion, optimizer=optimizer_mobile, device=device, num_epochs=3,
                                   batch_size=32, num_workers=12)
torch.save(model_mobile_trained.state_dict(), 'models/weights-cnn-baseline-mobile-v2.pt')

  0%|          | 0/5011 [00:00<?, ?it/s]

Epoch 1/3
----------


/home/yxlee245/ndsc-2019/src/model_api_pytorch.py:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  for c, attribute in zip(range(len(self.class_sizes)), self.attributes)}
  0%|          | 0/5011 [00:00<?, ?it/s]

Training Loss: 25.6787

Epoch 2/3
----------


  0%|          | 0/5011 [00:00<?, ?it/s]

Training Loss: 25.6566

Epoch 3/3
----------


100%|██████████| 5011/5011 [11:22<00:00, 13.11it/s]


Training Loss: 25.6477

Training completed in 36m 22s


## Generate Submissions

Load models

In [11]:
# Beauty
model_beauty_saved = ModifiedCNNv2(class_sizes=num_classes_beauty, attributes=attributes_beauty,
                                   model_pretrained=model_beauty_ft)
model_beauty_saved.set_multiple_gpus()
model_beauty_saved.to(device)
model_beauty_saved.load_state_dict(torch.load('models/weights-cnn-baseline-beauty-v2.pt'))

# Fashion
model_fashion_saved = ModifiedCNNv2(class_sizes=num_classes_fashion, attributes=attributes_fashion,
                                    model_pretrained=model_fashion_ft)
model_fashion_saved.set_multiple_gpus()
model_fashion_saved.to(device)
model_fashion_saved.load_state_dict(torch.load('models/weights-cnn-baseline-fashion-v2.pt'))

# Mobile
model_mobile_saved = ModifiedCNNv2(class_sizes=num_classes_mobile, attributes=attributes_mobile,
                                   model_pretrained=model_mobile_ft)
model_mobile_saved.set_multiple_gpus()
model_mobile_saved.to(device)
model_mobile_saved.load_state_dict(torch.load('models/weights-cnn-baseline-mobile-v2.pt'))

Define transforms for validation set

In [12]:
transforms_val = transforms.Compose([
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

Load in validation sets

In [13]:
df_beauty_val = pd.read_csv('data/raw/beauty_data_info_val_competition.csv')
df_fashion_val = pd.read_csv('data/raw/fashion_data_info_val_competition.csv')
df_mobile_val = pd.read_csv('data/raw/mobile_data_info_val_competition.csv')

Create dataframes for submission

In [17]:
df_beauty_submission = make_submission(df_val=df_beauty_val, model=model_beauty_saved, transforms=transforms_val,
                                       root_dir='data/raw', le_list=le_beauty_list)
df_beauty_submission.to_csv('data/derived/submission_cnn_baseline_beauty_v2.csv', index=False)

  0%|          | 0/76545 [00:00<?, ?it/s]/home/yxlee245/ndsc-2019/src/model_api_pytorch.py:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  for c, attribute in zip(range(len(self.class_sizes)), self.attributes)}
100%|██████████| 76545/76545 [2:00:14<00:00,  9.39it/s]  


In [18]:
df_fashion_submission = make_submission(df_val=df_fashion_val, model=model_fashion_saved, transforms=transforms_val,
                                        root_dir='data/raw', le_list=le_fashion_list)
df_fashion_submission.to_csv('data/derived/submission_cnn_baseline_fashion_v2.csv', index=False)

  0%|          | 0/69498 [00:00<?, ?it/s]/home/yxlee245/ndsc-2019/src/model_api_pytorch.py:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  for c, attribute in zip(range(len(self.class_sizes)), self.attributes)}
100%|██████████| 69498/69498 [2:03:54<00:00,  8.00it/s]  


In [14]:
df_mobile_submission = make_submission(df_val=df_mobile_val, model=model_mobile_saved, transforms=transforms_val,
                                       root_dir='data/raw', le_list=le_mobile_list)
df_mobile_submission.to_csv('data/derived/submission_cnn_baseline_mobile_v2.csv', index=False)

  0%|          | 0/40417 [00:00<?, ?it/s]/home/yxlee245/ndsc-2019/src/model_api_pytorch.py:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  for c, attribute in zip(range(len(self.class_sizes)), self.attributes)}
100%|██████████| 40417/40417 [1:02:05<00:00,  7.78it/s]


In [16]:
df_beauty_submission = pd.read_csv('data/derived/submission_cnn_baseline_beauty_v2.csv')
df_fashion_submission = pd.read_csv('data/derived/submission_cnn_baseline_fashion_v2.csv')
df_mobile_submission = pd.read_csv('data/derived/submission_cnn_baseline_mobile_v2.csv')
df_submissions = pd.concat((df_beauty_submission, df_fashion_submission, df_mobile_submission), ignore_index=True)
df_submissions.to_csv('data/submissions/submissions_cnn_baseline_20190312.csv', index=False)

Kaggle Score: 0.11710